## Logistic Regression Classification Algorithm

In [1]:
import numpy as np

class LogisticRegression:
    def __init__(self, learning_rate=0.01, max_iter=1000):
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.weights = None

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def log_likelihood(self, X, y, weights):
        scores = np.dot(X, weights)
        ll = np.sum(y * scores - np.log(1 + np.exp(scores)))
        return ll

    def gradient_ascent(self, X, y):
        # Add intercept term to X
        intercept = np.ones((X.shape[0], 1))
        X = np.hstack((intercept, X))
        
        # Initialize weights
        weights = np.zeros(X.shape[1])
        
        # Iterate to perform gradient ascent
        for step in range(self.max_iter):
            predictions = self.sigmoid(np.dot(X, weights))
            # Update weights with gradient
            output_error_signal = y - predictions
            gradient = np.dot(X.T, output_error_signal)
            weights += self.learning_rate * gradient
            
            # Print log-likelihood every so often
            if step % 100 == 0:
                print(f'Log-Likelihood at step {step}: {self.log_likelihood(X, y, weights)}')
        
        return weights

    def fit(self, X, y):
        self.weights = self.gradient_ascent(X, y)

    def predict_proba(self, X):
        intercept = np.ones((X.shape[0], 1))
        X = np.hstack((intercept, X))
        return self.sigmoid(np.dot(X, self.weights))

    def predict(self, X):
        probabilities = self.predict_proba(X)
        return (probabilities >= 0.5).astype(int)

# Example usage
# Generate some synthetic data
np.random.seed(0)
X = np.random.randn(100, 3)
y = (np.dot(X, [1, 2, 3]) + 1 > 0).astype(int)  # Generate labels based on a decision boundary

# Create and train the model
model = LogisticRegression(learning_rate=0.1, max_iter=500)
model.fit(X, y)

# Predict new data
test_X = np.random.randn(5, 3)
predictions = model.predict(test_X)
print("Predictions:", predictions)


Log-Likelihood at step 0: -14.79505628460326
Log-Likelihood at step 100: -2.5597551641860514
Log-Likelihood at step 200: -1.7043325020018272
Log-Likelihood at step 300: -1.3202303412571372
Log-Likelihood at step 400: -1.0927174320690685
Predictions: [1 1 0 1 0]


In [6]:
X = np.array([[40], [55], [30], [70]])  # Income of each person
y = np.array([0, 1, 0, 1])  # Purchase outcome, 0 = no, 1 = yes

# Step 2: Create and train the logistic regression model
model = LogisticRegression(learning_rate=0.0001, max_iter=1000000)
model.fit(X, y)

# Step 3: Make a prediction for an income of 65
test_X = np.array([[65]])  # The income value for which we want to predict
prediction = model.predict(test_X)

# Output the prediction
print("Prediction for income 65:", "Purchase" if prediction[0] == 1 else "No Purchase")

Log-Likelihood at step 0: -2.70680774609775
Log-Likelihood at step 100: -2.6224760822071174
Log-Likelihood at step 200: -2.6198159637507863
Log-Likelihood at step 300: -2.6171602464418364
Log-Likelihood at step 400: -2.6145089232982865
Log-Likelihood at step 500: -2.611861987339901
Log-Likelihood at step 600: -2.6092194315882544
Log-Likelihood at step 700: -2.6065812490667906
Log-Likelihood at step 800: -2.60394743280089
Log-Likelihood at step 900: -2.6013179758179263
Log-Likelihood at step 1000: -2.5986928711473296
Log-Likelihood at step 1100: -2.5960721118206482
Log-Likelihood at step 1200: -2.5934556908716107
Log-Likelihood at step 1300: -2.590843601336178
Log-Likelihood at step 1400: -2.588235836252618
Log-Likelihood at step 1500: -2.5856323886615504
Log-Likelihood at step 1600: -2.583033251606017
Log-Likelihood at step 1700: -2.580438418131534
Log-Likelihood at step 1800: -2.5778478812861536
Log-Likelihood at step 1900: -2.5752616341205226
Log-Likelihood at step 2000: -2.572679669

### Logistic Regression with Regularization

In [8]:
import numpy as np

class RegularizedLogisticRegression:
    def __init__(self, learning_rate=0.01, max_iter=1000, regularization='l2', C=1.0):
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.regularization = regularization
        self.C = C  # Regularization strength (smaller values specify stronger regularization)
        self.weights = None

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def compute_cost(self, X, y, weights):
        """Compute the cost function, optionally including L1 or L2 regularization."""
        m = X.shape[0]
        predictions = self.sigmoid(np.dot(X, weights))
        log_loss = - np.sum(y * np.log(predictions + 1e-9) + (1 - y) * np.log(1 - predictions + 1e-9)) / m
        
        if self.regularization == 'l2':
            l2_penalty = (self.C / 2) * np.sum(np.square(weights[1:]))
            return log_loss + l2_penalty
        elif self.regularization == 'l1':
            l1_penalty = self.C * np.sum(np.abs(weights[1:]))
            return log_loss + l1_penalty
        else:
            return log_loss

    def gradient_descent(self, X, y):
        """Perform gradient descent, including L1 or L2 regularization adjustments."""
        m, n = X.shape
        weights = np.zeros(n)
        for step in range(self.max_iter):
            predictions = self.sigmoid(np.dot(X, weights))
            errors = predictions - y
            gradient = np.dot(X.T, errors) / m
            
            if self.regularization == 'l2':
                # Exclude bias term from regularization
                gradient[1:] += self.C * weights[1:] / m
            elif self.regularization == 'l1':
                # Compute subgradient for L1
                gradient[1:] += self.C * np.sign(weights[1:]) / m
            
            weights -= self.learning_rate * gradient
            
            if step % 100 == 0:
                cost = self.compute_cost(X, y, weights)
                print(f"Cost at step {step}: {cost}")
                
        return weights

    def fit(self, X, y):
        """Fit the logistic regression model to the data."""
        intercept = np.ones((X.shape[0], 1))
        X = np.hstack((intercept, X))
        self.weights = self.gradient_descent(X, y)

    def predict_proba(self, X):
        """Calculate the probability of the classes."""
        intercept = np.ones((X.shape[0], 1))
        X = np.hstack((intercept, X))
        return self.sigmoid(np.dot(X, self.weights))

    def predict(self, X):
        """Predict class labels for samples in X."""
        probabilities = self.predict_proba(X)
        return (probabilities >= 0.5).astype(int)

# Example usage
np.random.seed(0)
X = np.random.randn(100, 3)  # 100 samples, 3 features
y = np.random.randint(0, 2, size=100)  # Binary targets

model = RegularizedLogisticRegression(learning_rate=0.1, max_iter=1000, regularization='l2', C=0.1)
model.fit(X, y)
predictions = model.predict(X)
print("Predictions:", predictions)

Cost at step 0: 0.6927271279971412
Cost at step 100: 0.6870496120496606
Cost at step 200: 0.6877542401811778
Cost at step 300: 0.6878748376265419
Cost at step 400: 0.687892936847626
Cost at step 500: 0.6878956706047441
Cost at step 600: 0.6878960892186421
Cost at step 700: 0.687896153941847
Cost at step 800: 0.6878961640099109
Cost at step 900: 0.6878961655819738
Predictions: [0 1 1 0 0 1 1 1 1 0 1 0 1 1 0 0 0 1 0 1 1 0 1 0 0 1 1 1 0 0 0 0 0 1 1 1 0
 0 0 0 0 1 0 0 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 0 0 1 0 1 0 0 0 0 1
 1 0 1 0 0 0 0 1 1 0 0 1 1 0 0 0 0 1 1 0 0 0 1 1 0 0]
